## Kindle review sentiment analysis

BEST PRACTICES :
1) Preprocessing and cleaning
2) Train test split
3) BOW,TF-IDF,Word2Vec
4) Train ML algorithms

In [1]:
import pandas as pd

In [2]:
data=pd.read_csv("all_kindle_review.csv")
data.head()

,Unnamed: 0.1,Unnamed: 0,asin,helpful,rating,reviewText,reviewTime,reviewerID,reviewerName,summary,unixReviewTime
0,0,11539,B0033UV8HI,"[8, 10]",3,"Jace Rankin may be short, but he's nothing to ...","09 2, 2010",A3HHXRELK8BHQG,Ridley,Entertaining But Average,1283385600
1,1,5957,B002HJV4DE,"[1, 1]",5,Great short read. I didn't want to put it dow...,"10 8, 2013",A2RGNZ0TRF578I,Holly Butler,Terrific menage scenes!,1381190400
2,2,9146,B002ZG96I4,"[0, 0]",3,I'll start by saying this is the first of four...,"04 11, 2014",A3S0H2HV6U1I7F,Merissa,Snapdragon Alley,1397174400
3,3,7038,B002QHWOEU,"[1, 3]",3,Aggie is Angela Lansbury who carries pocketboo...,"07 5, 2014",AC4OQW3GZ919J,Cleargrace,very light murder cozy,1404518400
4,4,1776,B001A06VJ8,"[0, 1]",4,I did not expect this type of book to be in li...,"12 31, 2012",A3C9V987IQHOQD,Rjostler,Book,1356912000


In [3]:
df=data[['reviewText','rating']]
df.head()

,reviewText,rating
0,"Jace Rankin may be short, but he's nothing to ...",3
1,Great short read. I didn't want to put it dow...,5
2,I'll start by saying this is the first of four...,3
3,Aggie is Angela Lansbury who carries pocketboo...,3
4,I did not expect this type of book to be in li...,4


In [4]:
df.shape

(12000, 2)

In [5]:
# check missing values
df.isnull().sum()

reviewText    0
rating        0
dtype: int64

In [6]:
df['rating'].unique()

array([3, 5, 4, 2, 1])

In [7]:
df['rating'].value_counts()

rating
5    3000
4    3000
3    2000
2    2000
1    2000
Name: count, dtype: int64

In [8]:
import warnings
warnings.filterwarnings('ignore')

Preprocessing and Cleaning

Here we divide data set into 2 parts - positive review (>3) is 1 and rest negative reviews are 0

In [9]:
df['rating']=df['rating'].apply(lambda x: 0 if x<3 else 1)

In [10]:
df['rating'].value_counts()

rating
1    8000
0    4000
Name: count, dtype: int64

In [11]:
# 1) lower all the cases
df['reviewText']=df['reviewText'].str.lower()


In [12]:
df.head()

,reviewText,rating
0,"jace rankin may be short, but he's nothing to ...",1
1,great short read. i didn't want to put it dow...,1
2,i'll start by saying this is the first of four...,1
3,aggie is angela lansbury who carries pocketboo...,1
4,i did not expect this type of book to be in li...,1


In [13]:
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\drrak\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [14]:
import sys
!{sys.executable} -m pip install beautifulsoup4


In [15]:
from bs4 import BeautifulSoup

In [16]:
# removing special characters
df['reviewText']=df['reviewText'].apply(lambda x:re.sub('[^a-z A-Z 0-9]+','',x))

In [17]:
# remove stopwords
df['reviewText']=df['reviewText'].apply(lambda x:" ".join([y for y in x.split() if y not in stopwords.words('english')]))

In [25]:
# Remove url 
df['reviewText']=df['reviewText'].apply(lambda x: re.sub(r'(http|https|ftp|ssh)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', '' , str(x)))

In [26]:
# remove HTML tags
df['reviewText'] = df['reviewText'].apply(
    lambda x: BeautifulSoup(x, 'html.parser').get_text()
)


In [27]:
# remove any additional spaces
df['reviewText']=df['reviewText'].apply(lambda x:" ".join(x.split()))

In [28]:
df.head()

,reviewText,rating
0,jace rankin may short hes nothing mess man hau...,1
1,great short read didnt want put read one sitti...,1
2,ill start saying first four books wasnt expect...,1
3,aggie angela lansbury carries pocketbooks inst...,1
4,expect type book library pleased find price right,1


Applying lemmatizer

In [29]:
from nltk.stem import WordNetLemmatizer

In [30]:
lemma=WordNetLemmatizer()

In [31]:
def lemmatize_words(text):
    return " ".join([lemma.lemmatize(word) for word in text.split()])

In [32]:
df['reviewText']=df['reviewText'].apply(lambda x:lemmatize_words(x))

Train - test split

In [33]:
from sklearn.model_selection import train_test_split

In [34]:
X_train,X_test,y_train,y_test=train_test_split(df['reviewText'],df['rating'],test_size=0.20)

Applying BoW

In [35]:
from sklearn.feature_extraction.text import CountVectorizer
bow=CountVectorizer()
X_train_bow=bow.fit_transform(X_train).toarray()
X_test_bow=bow.transform(X_test).toarray()


Applying TF-IDF

In [36]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer()
X_train_tfidf=tfidf.fit_transform(X_train).toarray()
X_test_tfidf=tfidf.transform(X_test).toarray()

In [43]:
# Check shape
print("X_train_avgw2v shape:", X_train_tfidf.shape)
print("X_test_avgw2v shape:", X_test_tfidf.shape)

X_train_avgw2v shape: (9600, 37750)
X_test_avgw2v shape: (2400, 37750)


Applying AvgWord2Vec

In [37]:
!pip install tqdm

In [40]:
from gensim.models import Word2Vec

In [49]:
from gensim.models import Word2Vec
import numpy as np

# Train Word2Vec model
tokenized_sentences = [sentence.split() for sentence in X_train]
model = Word2Vec(sentences=tokenized_sentences, 
                 vector_size=100, 
                 window=5, 
                 min_count=1, 
                 workers=4)

# Function to convert sentence to average word vector
def avg_word2vec(sentence, model):
    words = sentence.split()
    word_vectors = [model.wv[word] for word in words if word in model.wv.index_to_key]
    if len(word_vectors) > 0:
        return np.mean(word_vectors, axis=0)
    else:
        return np.zeros(model.vector_size)

# Apply to training and test data
X_train_avgw2v = np.array([avg_word2vec(sentence, model) for sentence in X_train])
X_test_avgw2v = np.array([avg_word2vec(sentence, model) for sentence in X_test])

# Check shape
print("X_train_avgw2v shape:", X_train_avgw2v.shape)
print("X_test_avgw2v shape:", X_test_avgw2v.shape)

X_train_avgw2v shape: (9600, 100)
X_test_avgw2v shape: (2400, 100)


# Training ML model

Using Gaussian NB for sparse matrix

In [55]:
from sklearn.naive_bayes import GaussianNB

In [56]:
nb_model_bow=GaussianNB().fit(X_train_bow,y_train)
nb_model_tfidf=GaussianNB().fit(X_train_tfidf,y_train)
nb_model_avgWord2Vec=GaussianNB().fit(X_train_avgw2v,y_train)

In [57]:
from sklearn.metrics import confusion_matrix,accuracy_score,classification_report

In [58]:
y_pred_bow=nb_model_bow.predict(X_test_bow)

In [59]:
y_pred_tfidf=nb_model_tfidf.predict(X_test_tfidf)

In [60]:
y_pred_avgWord2Vec=nb_model_avgWord2Vec.predict(X_test_avgw2v)

BoW performance

In [61]:
confusion_matrix(y_test,y_pred_bow)

array([[524, 278],
       [703, 895]])

In [62]:
print("BOW accuracy: ",accuracy_score(y_test,y_pred_bow))

BOW accuracy:  0.59125


TF-IDF Performance

In [63]:
confusion_matrix(y_test,y_pred_tfidf)

array([[508, 294],
       [693, 905]])

In [64]:
print("TF-IDF accuracy: ",accuracy_score(y_test,y_pred_tfidf))

TF-IDF accuracy:  0.58875


AvgWord2Vec Performance

In [65]:
confusion_matrix(y_test,y_pred_avgWord2Vec)

array([[ 585,  217],
       [ 564, 1034]])

In [66]:
print("Average Word2Vec accuarcy: ",accuracy_score(y_test,y_pred_avgWord2Vec))

Average Word2Vec accuarcy:  0.6745833333333333
